In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import unicodedata
import time
from tqdm import tqdm
import re
import sys
import requests
import warnings
import pandas as pd

from time import sleep
from copy import deepcopy
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display

In [10]:
# Print Timestamp At time of crawl
datePosted = str(datetime.date.today())
print('Time of Crawl: ' + datePosted)

Time of Crawl: 2020-07-29


In [11]:
warnings.filterwarnings('ignore')

def printmd(string):
    display(Markdown(string))

printmd("<span style='font-family:courier;color:green;font-size:115%;' >**--> Libraries Imported Successfully.**</span>")

<span style='font-family:courier;color:green;font-size:115%;' >**--> Libraries Imported Successfully.**</span>

In [12]:
print("\n===========================")
print("Fetching University Page(s) :")
print("===========================")

base_url = 'https://researchportal.bath.ac.uk/en/persons/?format=&page='

total_pages = 10

uni_urls = []

for x in tqdm(range(total_pages)):
    uni_urls.append(base_url+str(x))
    
no_of_profiles = total_pages*50
    
printmd("<span style='font-family:courier;color:green;font-size:115%;' > **--> There are Total "+str(total_pages)+" university page(s) found, each page(s) contains 50 researchers profile links.**</span>")


Fetching University Page(s) :


<span style='font-family:courier;color:green;font-size:115%;' > **--> There are Total 10 university page(s) found, each page(s) contains 50 researchers profile links.**</span>

In [13]:
print("\n================================")
print("Fetching links of All Profiles : ")
print("================================")

prof_links = []

for uni_url in tqdm(uni_urls):
    request = requests.get(uni_url)
    soup = BeautifulSoup(request.content, 'html.parser')    
    result_grid = soup.find('section', { "class" : "grid-results"})
    
    for a in result_grid.find_all('a', { "class" : "link person" }, href=True):
        prof_links.append(a['href'])


printmd("<span style='font-family:courier;color:green;font-size:115%;' > **--> Total "+str(len(prof_links))+"  Numbers of Researchers Profile Found.**</span>")


Fetching links of All Profiles : 


<span style='font-family:courier;color:green;font-size:115%;' > **--> Total 500  Numbers of Researchers Profile Found.**</span>

In [14]:
print("\n=====================================")
print("Fetching Details of Each Researcher : ")
print("=====================================")

prof_data = []


for row in tqdm(prof_links):
    prof_request = requests.get(row)
    prof_soup = BeautifulSoup(prof_request.content, 'html.parser')
    prof_grid = prof_soup.find('div', { "class" : "profile"})
    
    info = dict()
    prof_info = []
    
    info['url'] = row if row else "-"
    info['image_url'] = "https://researchportal.bath.ac.uk/"+prof_grid.find('img').attrs['src'] if prof_grid.find('img').attrs['src'] else "-"
    info['name'] = prof_grid.select_one('.person-details h1').text.strip() if prof_grid.select_one('.person-details h1') else "-"
    info['department'] = prof_grid.select_one('ul li a').text.strip() if prof_grid.select_one('ul li a') else "-"
    info['university'] = "University of Bath"
    info['phone'] = prof_grid.select_one('.phones .description').text.strip() if prof_grid.select_one('.phones .description') else "-"
    info['email'] = prof_grid.select_one('.emails .description').text.strip() if prof_grid.select_one('.emails .description') else "-"
    info['fingerprints_url'] = row+"/fingerprints" if row else "-"
    info['projects_url'] = row+"/projects" if row else "-"
    
    prof_info.append(info["url"])
    prof_info.append(info["image_url"])
    prof_info.append(info["name"])
    prof_info.append(info["department"])
    prof_info.append(info["university"])
    prof_info.append(info["phone"])
    prof_info.append(info["email"])
    
    ##########################
    
    prof_info.append(info["fingerprints_url"])
    
    fingerprint_request = requests.get(info["fingerprints_url"])
    fingerprint_soup = BeautifulSoup(fingerprint_request.content, 'html.parser')    
    fingerprint_grid = fingerprint_soup.find('div', { "class" : "person-fingerprints"})
    
    prof_research = []
    
    for research in fingerprint_soup.find_all('span', { "class" : "concept" }):
        prof_research.append(research.text)
    
    prof_info.append(str(prof_research)[1:-1])
    
    ##########################
    
    prof_info.append(info["projects_url"])
    
    project_request = requests.get(info["projects_url"])
    project_soup = BeautifulSoup(project_request.content, 'html.parser')    
    project_grid = project_soup.find('section', { "class" : "list-results"})

    prof_all_project = []
    
    for a in project_grid.find_all('div', { "class" : "result-container" }):

        proj_info = dict()
        
        proj_info['title'] = a.select_one('.title').text.strip() if a.select_one('.title') else "-"
        proj_info['person'] = a.select_one('p span').text.strip() if a.select_one('p span') else "-"
        proj_info['fundings'] = a.select_one('.fundings').text.strip() if a.select_one('.fundings') else "-"
        proj_info['period'] = a.select_one('.period').text.strip() if a.select_one('.period') else "-"
        proj_info['type'] = a.select_one('.type_classification').text.strip() if a.select_one('.type_classification') else "-"

        prof_proj = []
        
        prof_proj.append(proj_info['title'])
        prof_proj.append(proj_info['person'])
        prof_proj.append(proj_info['fundings'])
        prof_proj.append(proj_info['period'])
        prof_proj.append(proj_info['type'])
        
        prof_all_project.append(prof_proj)
    
    prof_info.append(prof_all_project)
    
    ##########################
    
    
    prof_data.append(prof_info)
    

#print(prof_info)
#print(prof_data)

printmd("<span style='font-family:courier;color:green;font-size:115%;' > **--> Profile Details of "+str(len(prof_links))+" Researchers fetched Successfully.**</span>")


Fetching Details of Each Researcher : 


<span style='font-family:courier;color:green;font-size:115%;' > **--> Profile Details of 500 Researchers fetched Successfully.**</span>

In [15]:
prof_df = pd.DataFrame(prof_data, columns= ['profile_url', 'image_url', 'full_name', 'department', 'university', 'contact', 'email', 'fingerprints_url', 'research_topic', 'projects_url', 'project_info'])

prof_df.to_csv (r'C:\\Users\\Saurabh\\Desktop\\IR\\Shruti\\new\\Docs csv etc\\bath_uni_df.csv', index = False, header=True)

prof_df

,profile_url,image_url,full_name,department,university,contact,email,fingerprints_url,research_topic,projects_url,project_info
0,https://researchportal.bath.ac.uk/en/persons/r...,https://researchportal.bath.ac.uk//assets/no-p...,Rahmi _,Department of Education,University of Bath,-,Rahmi@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/r...,,https://researchportal.bath.ac.uk/en/persons/r...,[]
1,https://researchportal.bath.ac.uk/en/persons/a...,https://researchportal.bath.ac.uk//files-asset...,"Andrea Abbas, Professor of the Sociology of Hi...",Department of Education,University of Bath,+44 (0) 1225 385217,A.Abbas@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/a...,"'sociology', 'pedagogics', 'university', 'soci...",https://researchportal.bath.ac.uk/en/persons/a...,[[Developing Leadership and Pedagogical Knowle...
2,https://researchportal.bath.ac.uk/en/persons/j...,https://researchportal.bath.ac.uk//files-asset...,Joan Abbas,Department of Social & Policy Sciences,University of Bath,-,J.E.Abbas@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/j...,"'pedagogics', 'Teaching', 'candidacy', 'educat...",https://researchportal.bath.ac.uk/en/persons/j...,[[Contemporary welfare state reforms: social i...
3,https://researchportal.bath.ac.uk/en/persons/t...,https://researchportal.bath.ac.uk//assets/no-p...,Taghried Abdelmagid,Department of Architecture & Civil Engineering,University of Bath,-,T.I.M.Abdelmagid@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/t...,,https://researchportal.bath.ac.uk/en/persons/t...,[]
4,https://researchportal.bath.ac.uk/en/persons/k...,https://researchportal.bath.ac.uk//assets/no-p...,Katty Abdelmalek,Management,University of Bath,-,K.S.N.Abdelmalek@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/k...,,https://researchportal.bath.ac.uk/en/persons/k...,[]
...,...,...,...,...,...,...,...,...,...,...,...
495,https://researchportal.bath.ac.uk/en/persons/y...,https://researchportal.bath.ac.uk//assets/no-p...,Yanyan Cheng,Department of Pharmacy & Pharmacology,University of Bath,-,Y.Cheng@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/y...,,https://researchportal.bath.ac.uk/en/persons/y...,[]
496,https://researchportal.bath.ac.uk/en/persons/k...,https://researchportal.bath.ac.uk//files-asset...,Kirill Cherednichenko,Department of Mathematical Sciences,University of Bath,+44 (0) 1225 386891,K.Cherednichenko@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/k...,"'Periodic Coefficients', 'Composite Media', 'H...",https://researchportal.bath.ac.uk/en/persons/k...,[[From functional models of operator theory to...
497,https://researchportal.bath.ac.uk/en/persons/r...,https://researchportal.bath.ac.uk//assets/no-p...,Robert Cherry,Department of Mechanical Engineering,University of Bath,-,rwc34@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/r...,,https://researchportal.bath.ac.uk/en/persons/r...,[]
498,https://researchportal.bath.ac.uk/en/persons/j...,https://researchportal.bath.ac.uk//files-asset...,John Chew,Department of Chemical Engineering,University of Bath,+44 (0) 1225 386132,Y.M.Chew@bath.ac.uk,https://researchportal.bath.ac.uk/en/persons/j...,"'Gaging', 'Fluid dynamics', 'Fouling', 'Cleani...",https://researchportal.bath.ac.uk/en/persons/j...,[[Computational fluid dynamics as a means to r...
